In [0]:
from pathlib import Path
import torch.nn

# Data

try to shuffel your data in training set. Random sampling

We want our training set to be in a random order, and that order should differ each iteration. But the validation set shouldn't be randomized.

PyTorch's defaults work fine for most things however:

In [0]:
# train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
# valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [0]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [0]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

#Callbacks

In [0]:
class Callback():
    _order = 0
    
    def set_runner(self, run): self.run = run
    def __getattr__(self, k): return getattr(self.run, k)

    def begin_fit(self):
        return True
    def after_fit(self): return True
    def begin_epoch(self):
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self):
        return True
    def after_loss(self):
        return True
    def after_backward(self): return True
    def after_step(self): return True
        

In [0]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)
    
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()

    def after_batch(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)

    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)
    

# Training

a basic optimizer

In [0]:
# class Optimizer():
  
#   def __init__(self, params, lr=0.5): self.params,self.lr=list(params),lr
    
#   def step(self):
#     with torch.no_grad():
#       for p in self.params: p -= p.grad*lr
        
#   def zero_grad(self):
#     for p in self.params: p.grad.data.zero_()

# Metrics

In [0]:
def accuracy(out, yb): return (torch.argmax(out,dim=1)==yb).float().mean()

In [0]:
#pass all the metrics ass a list
class AvgStats():
  def __init__(self, metrics, in_train): self.metrics, self.in_train = metrics, in_train
    
  def reset(self):
    self.tot_loss, self.count = 0., 0
    self.tot_mets = [0.]*len(self.metrics)
    
  @property
  def all_stats(self): return [self.tot_loss] + self.tot_mets
  
  @property
  def avg_stats(self): return [o/self.count for o in self.all_stats]
  
  def __repr__(self):
    if not self.count: return ""
    return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"
  
  def accumulate(self, run):
      bn = run.xb.shape[0]
      self.tot_loss += run.loss*bn
      self.tot_mets += [metric(run.pred,run.yb) for metric in self.metrics]*bn
      self.count += bn
